# STYLE GUIDE
### 본 문서는 PyTorch Lightning의 [공식 가이드](https://pytorch-lightning.readthedocs.io/en/latest/starter/style_guide.html)의 한글 번역본입니다. (옮긴이 [dnap512](https://github.com/dnap512))


Lightning의 주요 목표는 가독성과 재현성을 향상시키는 것입니다. GitHub 저장소를 살펴보고 Lightning 모듈과 관심 있는 항목을 찾기 위해 어디를 봐야 하는지 정확히 알고 있다고 상상해 보세요.

이 스타일 가이드의 목표는 Lightning 코드가 유사하게 구성되는 것입니다.

---

## LightningModule

다음은 LightningModule 구성에 대한 모범 사례입니다.

### System과 Model
![시스템](https://pl-bolts-doc-images.s3.us-east-2.amazonaws.com/pl_docs/model_system.png)


LightningModule의 기본 원칙은 전체 시스템이 독립적이어야 한다는 것입니다. Lightning에서는 시스템과 모델을 구분합니다.

모델은 resnet18, RNN 등과 같은 것입니다.

시스템은 모델들이 서로 상호 작용하는 방식을 정의합니다. 이에 대한 예는 다음과 같습니다.

- GAN
- Seq2Seq
- BERT
- etc

LightningModule은 시스템과 모델을 모두 정의할 수 있습니다.

다음은 모델을 정의하는 LightningModule입니다.

In [ ]:
class LitModel(LightningModule):
    def __init__(self, num_layers: int = 3):
        super().__init__()
        self.layer_1 = nn.Linear()
        self.layer_2 = nn.Linear()
        self.layer_3 = nn.Linear()

다음은 시스템을 정의하는 LightningModule입니다.

In [ ]:
class LitModel(LightningModule):
    def __init__(self, encoder: nn.Module = None, decoder: nn.Module = None):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

빠른 프로토타이핑을 위해 LightningModule에서 모든 계산을 정의하는 것이 종종 유용합니다. 재사용성과 확장성을 위해 관련 백본을 전달하는 것이 더 나을 수 있습니다.



#### Method order

LightningModule에서 필요한 메서드는 다음과 같습니다.

- init
- training_step
- configure_optimizers

하지만, 여러분이 구현할 수 있는 옵션은 더 많습니다. 추천드리는 것들은 다음과 같네요.

- model/system definition (init)
- if doing inference, define forward
- training hooks
- validation hooks
- test hooks
- configure_optimizers
- any other hooks

실제로 다음 코드같이 생겼습니다.

In [ ]:
class LitModel(pl.LightningModule):

    def __init__(...):

    def forward(...):

    def training_step(...)

    def training_step_end(...)

    def training_epoch_end(...)

    def validation_step(...)

    def validation_step_end(...)

    def validation_epoch_end(...)

    def test_step(...)

    def test_step_end(...)

    def test_epoch_end(...)

    def configure_optimizers(...)

    def any_extra_hook(...)

#### Forward와 Training_step

추론/예측을 위해 forward를 사용하고 training_step을 독립적으로 유지하는 것이 좋습니다.

In [ ]:
def forward(...):
    embeddings = self.encoder(x)

def training_step(...):
    x, y = ...
    z = self.encoder(x)
    pred = self.decoder(z)
    ...

하지만 DataParallel을 사용하는 경우 수동으로 forward를 호출해야 합니다.

In [ ]:
def training_step(...):
    x, y = ...
    z = self(x)  # < ---------- instead of self.encoder(x)
    pred = self.decoder(z)
    ...

---

## Data

다음은 데이터 처리를 위한 좋은 예입니다.

### Dataloaders

Lightning은 데이터로더를 사용하여 시스템을 통한 모든 데이터 흐름을 처리합니다. 데이터로더를 구성할 때마다 최대 효율성을 위해 `num_workers`를 조정해야 합니다.

> <span style='color:red'>경고:</span> num_workers > 0과 함께 ddp_spawn을 사용하지 않도록 하십시오. 그렇지 않으면 코드에 병목 현상이 발생합니다.

### DataModules

Lightning은 데이터 모듈을 도입했습니다. 데이터 로더의 문제는 다음 질문에 모두 답해야 하기 때문에 전체 데이터셋를 공유하는 것이 여전히 어려운 경우가 많다는 것입니다.

- 어떤 Split이 사용되었습니까?
- 이 데이터셋에는 몇 개의 샘플이 있습니까?
- 어떤 transfroms가 사용되었습니까?
- 기타 등등...

이러한 이유로 데이터 모듈을 사용하는 것이 좋습니다. 이는 팀의 시간도 많이 절약할 수 있기 때문에 협업할 때 특히 중요합니다.

그들이 해야 할 일은 데이터 모듈을 Lightning Trainer에 떨어뜨리고 데이터에 수행된 작업에 대해 걱정하지 않는 것뿐입니다.

이는 Data cleaning 및 ad-hoc instructions로 인해 아이디어를 반복하는 과정이 느려지는 연구나 기업 환경 모두에 해당됩니다.